# Generate templates and recordings on multiple probes

This notebook demonstrates how to generate templates and recordings with the same neurons and spiking activity, but on different probes. This can be useful to characterize how probe geometry affects spike sorting performance

In [ ]:
import MEArec as mr
import MEAutility as mu
import matplotlib.pyplot as plt
from pprint import pprint
from pathlib import Path
import numpy as np
from distutils.version import StrictVersion

%matplotlib notebook

In [ ]:
if StrictVersion(mr.__version__) < StrictVersion('1.6.0'):
    raise Exception("This feature is available in MEArec >= 1.6")

## Generate templates

Let's first choose our two probes:

In [ ]:
mr.available_probes()

In [ ]:
probe1_name = 'Neuronexus-32'
probe2_name = 'Neuropixels-24'

In [ ]:
probe1 = mu.return_mea(probe1_name)
probe2 = mu.return_mea(probe2_name)

In [ ]:
fig1, axs = plt.subplots(ncols=2)

mu.plot_probe(probe1, ax=axs[0])
mu.plot_probe(probe2, ax=axs[1])

axs[0].set_ylim(-400, 400)
axs[1].set_ylim(-400, 400)
axs[0].set_title(probe1_name)
axs[1].set_title(probe2_name)

Now we can generate templates on the first probe. We'll use a small `n` to speed up the simulation:

In [ ]:
templates_params = mr.get_default_templates_params()
cell_models_folder = mr.get_default_cell_models_folder()

In [ ]:
templates_params['probe'] = probe1_name
templates_params['n'] = 5
# optionally you can set the seed: templates_params['seed'] = 0

In [ ]:
tempgen1 = mr.gen_templates(cell_models_folder=cell_models_folder, params=templates_params, verbose=True)

Now we can use the `tempgen1` to generate templates on the second probe. Passing the `tempgen` argument will bypass the random location/rotation of the cell model, and instead use the ones already simulated.

We just need to update the params with the second probe:

In [ ]:
templates_params['probe'] = probe2_name

In [ ]:
tempgen2 = mr.gen_templates(cell_models_folder=cell_models_folder, params=templates_params, 
                            tempgen=tempgen1, verbose=True)

Let's now check that the locations are the same!

In [ ]:
for i in np.arange(len(tempgen1.locations)):
    print(f"Template {i}: loc1 - {tempgen1.locations[i]} -- loc2 - {tempgen2.locations[i]}")

Let's see how the templates look:

In [ ]:
fig2, axs = plt.subplots(ncols=2)

mr.plot_templates(tempgen1, template_ids=[0, 1, 2, 3], ax=axs[0])
mr.plot_templates(tempgen2, template_ids=[0, 1, 2, 3], ax=axs[1])

Great! Now that we have matched templates on two probes, let's generate some recordings!

## Generate recordings

In [ ]:
recording_params = mr.get_default_recordings_params()

Let's have 5 cells and a duration of 10 s:

In [ ]:
recording_params["spiketrains"]["duration"] = 10
recording_params["spiketrains"]["n_exc"] = 4
recording_params["spiketrains"]["n_inh"] = 1

In [ ]:
# let's set the seeds to get the same spiking activity and convolution modulation
recording_params["seeds"]["spiketrains"] = 0
recording_params["seeds"]["templates"] = 1
recording_params["seeds"]["convolution"] = 2

In [ ]:
recording_params["recordings"]["noise_level"] = 10
recording_params["recordings"]["filter"] = False

Ready to generate the first recording:

In [ ]:
recgen1 = mr.gen_recordings(params=recording_params, tempgen=tempgen1, verbose=True)

The `recgen` object has a `template_ids` field that contains the template indices from the `TemplateGenerator`. Since we generated templates on the second probe with the same templates, we can use these indices to force the same selection:

In [ ]:
recgen1.template_ids

In [ ]:
recgen2 = mr.gen_recordings(params=recording_params, tempgen=tempgen2, template_ids=recgen1.template_ids,
                            verbose=True)

Let's take a look at the recordings:

In [ ]:
fig2, axs = plt.subplots(ncols=2, figsize=(10, 8))

_ = mr.plot_recordings(recgen1, lw=0.1, overlay_templates=True,
                       max_channels_per_template=1, cmap='rainbow', templates_lw=0.3,
                       start_time=0, end_time=5, ax=axs[0])

_ = mr.plot_recordings(recgen2, lw=0.1, overlay_templates=True,
                       max_channels_per_template=1, cmap='rainbow', templates_lw=0.3,
                       start_time=0, end_time=5, ax=axs[1])

We have now the same spiking activity generated by the same neuronal models on 2 different probes!